In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
import sys
sys.path.insert(1, '../src/')

from image_tools import *
from text_tools import *
from data_prepare import *
import keras
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import matplotlib.pyplot as plt

In [2]:
data_path = "../../ndsc_data/"
path_image = '../../ndsc_data/training_img/training_img/'

In [3]:
d_train = pd.read_csv(os.path.join(data_path, "new_training_set.csv"),
                      usecols=["title_1", "image_1", "title_2", "image_2", "Label"])
d_test = pd.read_csv(os.path.join(data_path, "new_test_sample.csv"))

In [4]:
d_train.shape

(10181, 5)

In [5]:
d_train.head()

,title_1,image_1,title_2,image_2,Label
0,Johnson’s ® Top to Toe Hair & Body Bath 500ml,fdff8b9b8229da091dd7d070aae05f81.jpg,Johnson's cottontouch top to toe hair & body b...,41e191742760932598c7bd201e5dad47.jpg,0
1,Sandal Humble,906cc44f0be72d4e767669b5b63e3a17.jpg,Sandal Humble Glass - Glanzton,7a556b836bfdd08ea592216440524a34.jpg,0
2,PROMO LIKUID LIKUIT LIQUIT BABY POD LIQUID SAL...,475c26635de18b9f93032400732ff336.jpg,Voporizer Liquit - Likuit - Likuid - Liquid Pr...,ace93bec689f3f1565800c500a8341fa.jpg,0
3,6 Pasang / Set Anting Tusuk Bentuk Lingkaran A...,e630997f6217555d6026547ad1c15f0b.jpg,Subei 6 Pasang / Set Anting Tusuk Boho Bohemia...,31abbc176b09f5bd1728cfc3ecbbfb9c.jpg,0
4,ROREC NATURAL SKIN CARE MASK ROREC SHEET MASK ...,a27d11700a7902febd039dc3a96f10f2.jpg,Rorec 86 Natural Skin Care Shert Mask All Variant,813ad9dd638c10f1765db9dde20c9e42.jpg,1


## Prepare Dataset

In [6]:
d_train2 = d_train[:10]
d_test2 = d_test[:10]

In [7]:
d_train2['title_1_pre'] = d_train2.title_1.apply(text_cleansing)
d_train2['title_2_pre'] = d_train2.title_2.apply(text_cleansing)

d_test2['title_1_pre'] = d_test2.title_1.apply(text_cleansing)
d_test2['title_2_pre'] = d_test2.title_2.apply(text_cleansing)

<ipython-input-7-6fc9d2101629>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_train2['title_1_pre'] = d_train2.title_1.apply(text_cleansing)
<ipython-input-7-6fc9d2101629>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_train2['title_2_pre'] = d_train2.title_2.apply(text_cleansing)
<ipython-input-7-6fc9d2101629>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [8]:
d_train, d_test, word2idx, idx2word = data_text_prep(d_train2, d_test2)

/Users/alamhanz/anaconda/envs/joko/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/alamhanz/anaconda/envs/joko/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [9]:
dataset_train = ShopeeDataset(d_train, d_test, word2idx, idx2word)
dataset_test = ShopeeDataset(d_train, d_test, word2idx, idx2word, stage = 'val')

In [10]:
type(dataset)

data_prepare.ShopeeDataset

In [12]:
t1_encode, t2_encode, i1_scaled, i2_scaled, label = dataset.__getitem__(3)